In [1]:
import mlflow
from mlflow.tracking import MlflowClient

In [2]:
MLFLOW_URI = 'sqlite:///mlflow.db'

client = MlflowClient(tracking_uri = MLFLOW_URI)

In [30]:
filter_string = "name = 'car-price-prediction-xgbregressor'"

prod_version = ''
for ver in client.search_model_versions(filter_string=filter_string):
    if ver.current_stage == 'Production':
        prod_version = ver
        break

print(prod_version.run_id, prod_version.source)

b056e5a423ab4c6b98f852d8b4d253d3 /home/eugene/Documents/projects/car-price-evaluation/mlruns/1/b056e5a423ab4c6b98f852d8b4d253d3/artifacts/model


In [32]:
prod_version_source = prod_version.source
xgboost_model = mlflow.xgboost.load_model(prod_version_source)
xgboost_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [35]:
preprocessor_addition = "artifacts/preprocessor/preprocessor.bin"
prod_version_preprocessor_source = prod_version_source[:prod_version_source.index("artifacts")]+preprocessor_addition
print(prod_version_preprocessor_source)

/home/eugene/Documents/projects/car-price-evaluation/mlruns/1/b056e5a423ab4c6b98f852d8b4d253d3/artifacts/preprocessor/preprocessor.bin


In [36]:
dv = mlflow.artifacts.download_artifacts(artifact_uri=prod_version_preprocessor_source, dst_path="models")

In [37]:
dv

'/home/eugene/Documents/projects/car-price-evaluation/models/preprocessor.bin'

In [38]:
#from AWS
TRACKING_SERVER_HOST = "ec2-3-90-186-17.compute-1.amazonaws.com"
client_aws = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [39]:
filter_string = "name = 'car-price-prediction-xgbregressor'"

prod_version = ''
for ver in client_aws.search_model_versions(filter_string=filter_string):
    if ver.current_stage == 'Production':
        prod_version = ver
        break

print(prod_version.run_id, prod_version.source)

5040770d28ab41f3af231692ef2e3c66 s3://cpe-mlflow-artifacts/1/5040770d28ab41f3af231692ef2e3c66/artifacts/model


In [40]:
prod_version_source = prod_version.source
xgboost_model = mlflow.xgboost.load_model(prod_version_source)
xgboost_model

s3://cpe-mlflow-artifacts/1/5040770d28ab41f3af231692ef2e3c66/artifacts/model/model.xgb

ModuleNotFoundError: No module named 'boto3'